In [33]:
# Load responses from all participants. Run this at first

import os
import pandas as pd
import json
import ipywidgets as widgets
from ipywidgets import Button, HBox
from IPython.display import display, clear_output, HTML
from dotenv import load_dotenv
from pyairtable import Table

load_dotenv()

API_TOKEN = os.getenv("APITOKEN")
APP_ID = os.getenv("APPID")

ResponseAirtable = Table(API_TOKEN, APP_ID, os.getenv("ResponseVersion")).all()
ResponseDataList = [each['fields'] for each in ResponseAirtable if 'fields' in each]
ResponseDataFrame = pd.DataFrame(ResponseDataList)

In [45]:
SURVEY_VERSION = 2 # Define survey version

In [ ]:
# Load responses based on survey version. Please define survey version manually.

SurveyVersionAirtable = Table(API_TOKEN, APP_ID, os.getenv("SurveyVersion")).all()

PRE_STUDY_ID = -1
QUESTIONNAIRE_ID = -1
POST_STUDY_ID = -1

for each in SurveyVersionAirtable:
    if 'fields' in each:
        if '$SurveyConfigID' in each['fields']:
            if each['fields']['$SurveyConfigID'] == SURVEY_VERSION:
                PRE_STUDY_ID = each['fields']['PreStudyID']
                QUESTIONNAIRE_ID = each['fields']['QuestionnaireID']
                POST_STUDY_ID = each['fields']['PostStudyID']

PreStudyDataFrame = pd.DataFrame()
if PRE_STUDY_ID >= 0:
    PrestudyAirtable = Table(API_TOKEN, APP_ID, f"PreStudyV{PRE_STUDY_ID}").all()
    PreStudyDataList = [each['fields'] for each in PrestudyAirtable if 'fields' in each]
    PreStudyDataFrame = pd.DataFrame(PreStudyDataList)

QuestionnaireDataFrame = pd.DataFrame()
if QUESTIONNAIRE_ID >= 0:
    QuestionnaireAirtable = Table(API_TOKEN, APP_ID, f"QuestionnaireV{QUESTIONNAIRE_ID}").all()
    QuestionnaireDataList = [each['fields'] for each in QuestionnaireAirtable if 'fields' in each]
    QuestionnaireDataFrame = pd.DataFrame(QuestionnaireDataList)

PostStudyDataFrame = pd.DataFrame()
if POST_STUDY_ID >= 0:
    PostStudyAirtable = Table(API_TOKEN, APP_ID, f"PostStudyV{POST_STUDY_ID}").all()
    PostStudyDataList = [each['fields'] for each in PostStudyAirtable if 'fields' in each]
    PostStudyDataFrame = pd.DataFrame(PostStudyDataList)


FilteredResponse = ResponseDataFrame.loc[
    (ResponseDataFrame['PreStudy ID'] == str(PRE_STUDY_ID)) & 
    (ResponseDataFrame['Questionnaire ID'] == str(QUESTIONNAIRE_ID)) & 
    (ResponseDataFrame["PostStudy ID"] == str(POST_STUDY_ID))
    ]



def LoadResponse(ResponseStr):
    ResStrs = ResponseStr.replace('{\n','').replace('\n}', '').split('\n')
    ResList = []
    for each in ResStrs:
        ResList.append(each.split(':')[1])

    return ResList



PrestudyResponseDataFrame = pd.DataFrame()
PrestudyResponseDataFrame['Participant ID'] = None
for i in range(len(PreStudyDataFrame)):
    columnName = PreStudyDataFrame[PreStudyDataFrame['$QuestionID'] == int(i)]['QuestionString'] + " " + PreStudyDataFrame[PreStudyDataFrame['$QuestionID'] == int(i)]['ResponseConfig'].apply(lambda x: str(x).replace('{\n', ' (').replace('\n}', ')').replace('\n', ', '))
    PrestudyResponseDataFrame[columnName] = None

QuestionnaireResponseDataFrame = pd.DataFrame()
QuestionnaireResponseDataFrame['Participant ID'] = None
for i in range(len(QuestionnaireDataFrame)):
    columnName = QuestionnaireDataFrame[QuestionnaireDataFrame['$QuestionID'] == int(i)]['QuestionString'] + " " + QuestionnaireDataFrame[QuestionnaireDataFrame['$QuestionID'] == int(i)]['ResponseConfig'].apply(lambda x: str(x).replace('{\n', ' (').replace('\n}', ')').replace('\n', ', '))
    QuestionnaireResponseDataFrame[columnName] = None

PoststudyResponseDataFrame = pd.DataFrame()
PoststudyResponseDataFrame['Participant ID'] = None
for i in range(len(PostStudyDataFrame)):
    columnName = PostStudyDataFrame[PostStudyDataFrame['$QuestionID'] == int(i)]['QuestionString'] + " " + PostStudyDataFrame[PostStudyDataFrame['$QuestionID'] == int(i)]['ResponseConfig'].apply(lambda x: str(x).replace('{\n', ' (').replace('\n}', ')').replace('\n', ', '))
    PoststudyResponseDataFrame[columnName] = None

for index, row in FilteredResponse.iterrows():
    ID = str(row['$Participant ID'])

    PreSRes = LoadResponse(row['PreStudy Response'])
    QuestRes = LoadResponse(row['Questionnaire Response'])
    PostRes = LoadResponse(row['PostStudy Response'])

    CurrentPreRowIndex = len(PrestudyResponseDataFrame)
    new_pre_row = [ID] + PreSRes
    PrestudyResponseDataFrame.loc[CurrentPreRowIndex] = new_pre_row

    CurrentQuestRowIndex = len(QuestionnaireResponseDataFrame)
    new_quest_row = [ID] + QuestRes
    QuestionnaireResponseDataFrame.loc[CurrentQuestRowIndex] = new_quest_row

    CurrentPostRowIndex = len(PoststudyResponseDataFrame)
    new_post_row = [ID] + PostRes
    PoststudyResponseDataFrame.loc[CurrentPostRowIndex] = new_post_row


def Display_Prestudy_Response(change):
    clear_output(wait=True)
    display(BTN_Container, Output)
    display(PrestudyResponseDataFrame)

def Display_Questionnaire_Response(change):
    clear_output(wait=True)
    display(BTN_Container, Output)
    display(QuestionnaireResponseDataFrame)

def Display_Poststudy_Response(change):
    clear_output(wait=True)
    display(BTN_Container, Output)
    display(PoststudyResponseDataFrame)

def Save_Dataframes(change):
    PrestudyResponseDataFrame.to_csv('Saved_Prestudy_Response.csv', index=False)
    QuestionnaireResponseDataFrame.to_csv('Saved_Questionnaire_Response.csv', index=False)
    PoststudyResponseDataFrame.to_csv('Saved_Poststudy_Response.csv', index=False)
    display(widgets.HTML('<div style="color: green; font-size: 16px;">[System]: All dataframes are saved!</div>'))

def Clear_All(change):
    clear_output()
    

Pre_BTN = widgets.Button(description='PreStudy Responses')
Pre_BTN.layout = widgets.Layout(width='auto', height='40px', margin='10px', padding='5px 10px', border='2px solid black', font_size='16px')
Pre_BTN.on_click(Display_Prestudy_Response)

Quest_BTN = widgets.Button(description='Questionnaire Responses')
Quest_BTN.layout = widgets.Layout(width='auto', height='40px', margin='10px', padding='5px 10px', border='2px solid black', font_size='16px')
Quest_BTN.on_click(Display_Questionnaire_Response)

Post_BTN = widgets.Button(description='PostStudy Responses')
Post_BTN.layout = widgets.Layout(width='auto', height='40px', margin='10px', padding='5px 10px', border='2px solid black', font_size='16px')
Post_BTN.on_click(Display_Poststudy_Response)

Save_BTN = widgets.Button(description='Save Dataframes to CSV')
Save_BTN.layout = widgets.Layout(width='auto', height='40px', margin='10px', padding='5px 10px', border='2px solid green', font_size='16px')
Save_BTN.on_click(Save_Dataframes)

Close_BTN = widgets.Button(description='Clear Tab')
Close_BTN.layout = widgets.Layout(width='auto', height='40px', margin='10px', padding='5px 10px', border='2px solid red', font_size='16px')
Close_BTN.on_click(Clear_All)

Output = widgets.Output()

BTN_Container = widgets.HBox([Pre_BTN, Quest_BTN, Post_BTN, Save_BTN, Close_BTN])

display(BTN_Container, Output)
Pre_BTN.click()
